## Library Imports


In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import pandas as pd
sns.set_style('darkgrid')
plt.style.use('seaborn-v0_8-pastel')

import warnings
warnings.simplefilter('ignore')

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor

In [2]:
data = pd.read_csv("UrbanEdgeApparel.csv")
data.head()

,Order ID,Order Status,Order Date,Order Day of Week,Order Month,Order Year,Customer ID,Company ID,Product ID,Product Variant ID,Product Unit Selling Price,Product Quantity,Total Selling Price,Payment Status,Shipment ID,Shipment Number,Shipping Address Type,Shipping City,Shipping State,Shipping Postal Code,Shipping Country
0,104,Completed,6/6/2013,Thursday,June,2013.0,Cust_3161,Company_87239,Prod_5030,Prod_Var_50300,4.30,50,215.00,Received,271,271,Business,Washington,District Of Columbia,20011-2343,United States
1,104,Completed,6/6/2013,Thursday,June,2013.0,Cust_3161,Company_87239,Prod_70018,Prod_Var_70018,1.50,50,75.00,Received,271,271,Business,Washington,District Of Columbia,20011-2343,United States
2,107,Completed,6/6/2013,Thursday,June,2013.0,Cust_2040,Company_83024,Prod_1000,Prod_Var_10003,3.95,1,3.95,Received,438,438,Business,Lone Tree,Colorado,80124,United States
3,107,Completed,6/6/2013,Thursday,June,2013.0,Cust_2040,Company_83024,Prod_1000,Prod_Var_10004,3.95,1,3.95,Received,438,438,Business,Lone Tree,Colorado,80124,United States
4,107,Completed,6/6/2013,Thursday,June,2013.0,Cust_2040,Company_83024,Prod_1000,Prod_Var_10005,3.95,2,7.90,Received,438,438,Business,Lone Tree,Colorado,80124,United States
